In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [2]:
#import required libraries
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from itertools import product

In [4]:

X_train = pd.read_csv(r'./data/X_train_NHkHMNU.csv')
Y_train = pd.read_csv(r'./data/y_train_ZAN5mwg.csv')
X_test = pd.read_csv(r'./data/X_test_final.csv')

In [3]:
# After downloading the X_train/X_test/Y_train .csv files in your working directory:

X_train = pd.read_csv(r'C:\Users\abhim\OneDrive\Desktop\Term 2\Ensemble\Project\Data-20240219T230440Z-001\Data\X_train_NHkHMNU.csv')
Y_train = pd.read_csv(r'C:\Users\abhim\OneDrive\Desktop\Term 2\Ensemble\Project\Data-20240219T230440Z-001\Data\y_train_ZAN5mwg.csv')
X_test = pd.read_csv(r'C:\Users\abhim\OneDrive\Desktop\Term 2\Ensemble\Project\Data-20240219T230440Z-001\Data\X_test_final.csv')

In [5]:
#concat the X_train and Y_train
X_train = pd.concat([X_train, Y_train], axis=1)

In [6]:
X_train.head()

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET,ID,TARGET
0,1054,206,FR,0.210099,-0.427458,-0.606523,0.606523,NaN,0.692860,NaN,...,-0.556356,-0.790823,-0.283160,-1.069070,-0.063404,0.339041,0.124552,-0.002445,1054,0.028313
1,2049,501,FR,-0.022399,-1.003452,-0.022063,0.022063,-0.573520,-1.130838,0.573520,...,-0.770457,1.522331,0.828412,0.437419,1.831241,-0.659091,0.047114,-0.490365,2049,-0.112516
2,1924,687,FR,1.395035,1.978665,1.021305,-1.021305,-0.622021,-1.682587,0.622021,...,-0.313338,0.431134,0.487608,0.684884,0.114836,0.535974,0.743338,0.204952,1924,-0.180840
3,297,720,DE,-0.983324,-0.849198,-0.839586,0.839586,-0.270870,0.563230,0.270870,...,-0.507570,-0.499409,-0.236249,0.350938,-0.417514,0.911652,-0.296168,1.073948,297,-0.260356
4,1101,818,FR,0.143807,-0.617038,-0.924990,0.924990,NaN,0.990324,NaN,...,-0.424550,-1.088158,-1.011560,0.614338,0.729495,0.245109,1.526606,2.614378,1101,-0.071733


In [7]:
#sort the data in ascending order of DAY_ID
X_train = X_train.sort_values(by='DAY_ID')
X_train = X_train.reset_index(drop=True)

X_train.head()

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET,ID,TARGET
0,1057,0,FR,0.048418,-0.465067,-0.079296,0.079296,NaN,0.650577,NaN,...,-0.856204,-0.655528,-1.099226,0.279885,0.583874,-1.071996,-0.074767,0.798003,1057,0.108953
1,1179,1,FR,0.966997,1.222131,0.331356,-0.331356,0.405549,0.778627,-0.405549,...,-0.497520,-1.184018,-1.465608,1.349212,0.231602,1.480313,0.931562,0.822047,1179,-0.063369
2,1327,2,FR,-0.068972,-0.667390,-1.102015,1.102015,-1.080403,0.256736,1.080403,...,-0.372156,-0.623041,-0.926064,-0.148950,0.641235,1.802550,1.140920,0.900434,1327,2.575976
3,111,2,DE,-0.068972,-0.667390,-1.102015,1.102015,-1.080403,0.256736,1.080403,...,-0.372156,-0.623041,-0.926064,-0.148950,0.641235,1.802550,1.140920,0.900434,111,0.861270
4,2016,3,FR,-0.134670,-0.834564,-1.051716,1.051716,-1.881881,-0.612133,1.881881,...,-1.118297,0.244859,0.488650,-1.710888,-0.951057,0.440121,-0.064550,-0.032756,2016,0.068905


In [8]:
#split the X_train data to X_train_FR and X_train_DE based on the COUNTRY column
X_train_FR = X_train[X_train['COUNTRY'] == 'FR']
X_train_DE = X_train[X_train['COUNTRY'] == 'DE']

In [9]:
fill=['DE_RAIN','FR_RAIN','DE_WIND','FR_WIND','DE_TEMP','FR_TEMP']

In [10]:
from statsmodels.tsa.arima.model import ARIMA
import numpy as np

def fill_missing_values(df, columns_to_fill):
    # Loop over the columns to fill
    for column in columns_to_fill:
        # Get the data for this column
        data = df[column]

        # Find where the NaNs are
        mask = np.isnan(data)

        # Prepare the data for ARIMA
        train_data = data[~mask]
        test_data = data[mask]

        # Fit the ARIMA model
        model = ARIMA(train_data, order=(5,1,0))
        model_fit = model.fit()

        # Use the model to predict the missing values
        predictions = model_fit.predict(start=len(train_data), end=len(train_data)+len(test_data)-1)

        # Fill the missing values with the predictions
        data[mask] = predictions

        # Use forward fill for any remaining missing values
        data = data.fillna(method='ffill')

        # Use backward fill for any remaining missing values
        data = data.fillna(method='bfill')

        # Update the column in the DataFrame
        df[column] = data
    return df

In [11]:
fill_missing_values(X_train_FR, fill)
fill_missing_values(X_train_DE, fill)

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET,ID,TARGET
3,111,2,DE,-0.068972,-0.667390,-1.102015,1.102015,-1.080403,0.256736,1.080403,...,-0.372156,-0.623041,-0.926064,-0.148950,0.641235,1.802550,1.140920,0.900434,111,0.861270
5,800,3,DE,-0.134670,-0.834564,-1.051716,1.051716,-1.881881,-0.612133,1.881881,...,-1.118297,0.244859,0.488650,-1.710888,-0.951057,0.440121,-0.064550,-0.032756,800,7.138604
7,831,5,DE,-0.297850,-0.470371,0.144615,-0.144615,-1.208286,-1.811403,1.208286,...,-0.790071,0.333817,0.021868,-0.062187,1.459745,-0.117977,0.550433,0.781870,831,0.026374
9,779,7,DE,0.057599,-0.625625,0.002239,-0.002239,-0.676226,-0.745182,0.676226,...,-0.663419,0.098940,1.367421,-0.812164,0.954384,-0.379980,0.518459,-0.034642,779,-0.021227
10,841,8,DE,-0.282744,-0.700027,1.309253,-1.309253,0.248085,-2.226840,-0.248085,...,-0.307449,1.690497,1.584210,-0.441943,-0.382423,1.557325,0.146634,2.329924,841,0.042996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1480,282,1202,DE,-1.182561,-1.259840,-1.201004,1.201004,-1.616737,0.619501,1.616737,...,-0.997403,-0.595651,-0.777719,0.039450,1.411276,0.184767,0.257420,0.507079,282,-0.091774
1484,505,1207,DE,1.002463,0.516789,0.651648,-0.651648,0.728817,-0.820640,-0.728817,...,-0.997403,-0.595651,-0.777719,0.039450,1.411276,0.099209,1.750872,0.646905,505,-1.324555
1487,823,1208,DE,0.046568,-0.709011,0.427976,-0.427976,-0.578391,-1.678101,0.578391,...,2.661142,0.472212,0.630211,1.065727,-1.439105,0.291714,0.568479,0.395742,823,0.060380
1490,771,1212,DE,-0.046100,-0.520506,-0.270515,0.270515,-0.137917,-0.789824,0.137917,...,-0.237434,1.358211,1.123953,-0.707839,-0.308232,0.017778,0.072168,-0.160792,771,-0.144320


In [12]:
columns_to_interpolate = ['DE_FR_EXCHANGE', 'FR_DE_EXCHANGE', 'DE_NET_EXPORT', 'FR_NET_EXPORT', 'DE_NET_IMPORT', 'FR_NET_IMPORT']
#Interpolates missing values in specific columns of the given dataframes.
X_train_FR[columns_to_interpolate] = X_train_FR[columns_to_interpolate].interpolate(method='linear', limit_direction='both', axis=0)
X_train_DE[columns_to_interpolate] = X_train_DE[columns_to_interpolate].interpolate(method='linear', limit_direction='both', axis=0)

In [13]:
X_train_FR.head()

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET,ID,TARGET
0,1057,0,FR,0.048418,-0.465067,-0.079296,0.079296,0.405549,0.650577,-0.405549,...,-0.856204,-0.655528,-1.099226,0.279885,0.583874,-1.071996,-0.074767,0.798003,1057,0.108953
1,1179,1,FR,0.966997,1.222131,0.331356,-0.331356,0.405549,0.778627,-0.405549,...,-0.497520,-1.184018,-1.465608,1.349212,0.231602,1.480313,0.931562,0.822047,1179,-0.063369
2,1327,2,FR,-0.068972,-0.667390,-1.102015,1.102015,-1.080403,0.256736,1.080403,...,-0.372156,-0.623041,-0.926064,-0.148950,0.641235,1.802550,1.140920,0.900434,1327,2.575976
4,2016,3,FR,-0.134670,-0.834564,-1.051716,1.051716,-1.881881,-0.612133,1.881881,...,-1.118297,0.244859,0.488650,-1.710888,-0.951057,0.440121,-0.064550,-0.032756,2016,0.068905
6,2047,5,FR,-0.297850,-0.470371,0.144615,-0.144615,-1.208286,-1.811403,1.208286,...,-0.790071,0.333817,0.021868,-0.062187,1.459745,-0.117977,0.550433,0.781870,2047,1.031308


In [14]:
def calculate_fuel_cost(df):
    """
    Calculates the fuel cost for different countries and fuel types.

    Parameters:
    df (pandas.DataFrame): The input DataFrame containing the necessary columns.

    Returns:
    pandas.DataFrame: The DataFrame with additional columns for fuel costs.

    """
    for ctry, kind in product(['DE', 'FR'], ['COAL', 'GAS']):
        df[f'{ctry}_{kind}_COST'] = df[f'{kind}_RET'] * df[f'{ctry}_{kind}']
    df['DE_LIGNITE_COST'] = df['COAL_RET'] * df['DE_LIGNITE']
    return df

In [15]:
X_train_DE = calculate_fuel_cost(X_train_DE)
X_train_DE.head()

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,GAS_RET,COAL_RET,CARBON_RET,ID,TARGET,DE_COAL_COST,DE_GAS_COST,FR_COAL_COST,FR_GAS_COST,DE_LIGNITE_COST
3,111,2,DE,-0.068972,-0.667390,-1.102015,1.102015,-1.080403,0.256736,1.080403,...,1.802550,1.140920,0.900434,111,0.861270,-0.282610,2.213675,-0.874976,0.826113,-0.537791
5,800,3,DE,-0.134670,-0.834564,-1.051716,1.051716,-1.881881,-0.612133,1.881881,...,0.440121,-0.064550,-0.032756,800,7.138604,0.041018,0.699151,0.046394,0.030499,0.010931
7,831,5,DE,-0.297850,-0.470371,0.144615,-0.144615,-1.208286,-1.811403,1.208286,...,-0.117977,0.550433,0.781870,831,0.026374,-0.039670,-0.125035,-0.421666,-0.062324,-0.153950
9,779,7,DE,0.057599,-0.625625,0.002239,-0.002239,-0.676226,-0.745182,0.676226,...,-0.379980,0.518459,-0.034642,779,-0.021227,0.132404,-0.146745,-0.403380,-0.276365,0.056409
10,841,8,DE,-0.282744,-0.700027,1.309253,-1.309253,0.248085,-2.226840,-0.248085,...,1.557325,0.146634,2.329924,841,0.042996,-0.034382,1.487869,-0.110837,0.433819,-0.035941


In [16]:
X_train_FR = calculate_fuel_cost(X_train_FR)
X_train_FR.head()

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,GAS_RET,COAL_RET,CARBON_RET,ID,TARGET,DE_COAL_COST,DE_GAS_COST,FR_COAL_COST,FR_GAS_COST,DE_LIGNITE_COST
0,1057,0,FR,0.048418,-0.465067,-0.079296,0.079296,0.405549,0.650577,-0.405549,...,-1.071996,-0.074767,0.798003,1057,0.108953,-0.039966,-0.635235,-0.007118,0.280396,-0.076148
1,1179,1,FR,0.966997,1.222131,0.331356,-0.331356,0.405549,0.778627,-0.405549,...,1.480313,0.931562,0.822047,1179,-0.063369,0.615194,1.644639,-0.732682,2.947344,1.015751
2,1327,2,FR,-0.068972,-0.667390,-1.102015,1.102015,-1.080403,0.256736,1.080403,...,1.802550,1.140920,0.900434,1327,2.575976,-0.282610,2.213675,-0.874976,0.826113,-0.537791
4,2016,3,FR,-0.134670,-0.834564,-1.051716,1.051716,-1.881881,-0.612133,1.881881,...,0.440121,-0.064550,-0.032756,2016,0.068905,0.041018,0.699151,0.046394,0.030499,0.010931
6,2047,5,FR,-0.297850,-0.470371,0.144615,-0.144615,-1.208286,-1.811403,1.208286,...,-0.117977,0.550433,0.781870,2047,1.031308,-0.039670,-0.125035,-0.421666,-0.062324,-0.153950


In [17]:
carbon_footprint_weights = {'LIGNITE': 1.05, 'COAL': 0.85, 'GAS': 0.7}

In [18]:
def calculate_carbon_footprint(df, carbon_footprint_weights):
    """
    Calculates the carbon footprint for each country and fuel type based on the given dataframe and carbon footprint weights.
    """
    for ctry, kind in product(['DE', 'FR'], ['COAL', 'GAS']):
        df[f'{ctry}_{kind}_CARBON'] = carbon_footprint_weights[kind] * df['CARBON_RET'] * df[f'{ctry}_{kind}']
    df['DE_LIGNITE_CARBON'] = carbon_footprint_weights['LIGNITE'] * df['CARBON_RET'] * df['DE_LIGNITE']
    return df

In [19]:
X_train_FR = calculate_carbon_footprint(X_train_FR, carbon_footprint_weights)
X_train_FR.head()

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,DE_COAL_COST,DE_GAS_COST,FR_COAL_COST,FR_GAS_COST,DE_LIGNITE_COST,DE_COAL_CARBON,DE_GAS_CARBON,FR_COAL_CARBON,FR_GAS_CARBON,DE_LIGNITE_CARBON
0,1057,0,FR,0.048418,-0.465067,-0.079296,0.079296,0.405549,0.650577,-0.405549,...,-0.039966,-0.635235,-0.007118,0.280396,-0.076148,0.362577,0.331012,0.064572,-0.146111,0.853381
1,1179,1,FR,0.966997,1.222131,0.331356,-0.331356,0.405549,0.778627,-0.405549,...,0.615194,1.644639,-0.732682,2.947344,1.015751,0.461441,0.639310,-0.549565,1.145703,0.941156
2,1327,2,FR,-0.068972,-0.667390,-1.102015,1.102015,-1.080403,0.256736,1.080403,...,-0.282610,2.213675,-0.874976,0.826113,-0.537791,-0.189585,0.774063,-0.586965,0.288870,-0.445656
4,2016,3,FR,-0.134670,-0.834564,-1.051716,1.051716,-1.881881,-0.612133,1.881881,...,0.041018,0.699151,0.046394,0.030499,0.010931,0.017693,-0.036424,0.020011,-0.001589,0.005824
6,2047,5,FR,-0.297850,-0.470371,0.144615,-0.144615,-1.208286,-1.811403,1.208286,...,-0.039670,-0.125035,-0.421666,-0.062324,-0.153950,-0.047898,0.580054,-0.509118,0.289129,-0.229615


In [20]:
X_train_DE = calculate_carbon_footprint(X_train_DE, carbon_footprint_weights)
X_train_DE.head()

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,DE_COAL_COST,DE_GAS_COST,FR_COAL_COST,FR_GAS_COST,DE_LIGNITE_COST,DE_COAL_CARBON,DE_GAS_CARBON,FR_COAL_CARBON,FR_GAS_CARBON,DE_LIGNITE_CARBON
3,111,2,DE,-0.068972,-0.667390,-1.102015,1.102015,-1.080403,0.256736,1.080403,...,-0.282610,2.213675,-0.874976,0.826113,-0.537791,-0.189585,0.774063,-0.586965,0.288870,-0.445656
5,800,3,DE,-0.134670,-0.834564,-1.051716,1.051716,-1.881881,-0.612133,1.881881,...,0.041018,0.699151,0.046394,0.030499,0.010931,0.017693,-0.036424,0.020011,-0.001589,0.005824
7,831,5,DE,-0.297850,-0.470371,0.144615,-0.144615,-1.208286,-1.811403,1.208286,...,-0.039670,-0.125035,-0.421666,-0.062324,-0.153950,-0.047898,0.580054,-0.509118,0.289129,-0.229615
9,779,7,DE,0.057599,-0.625625,0.002239,-0.002239,-0.676226,-0.745182,0.676226,...,0.132404,-0.146745,-0.403380,-0.276365,0.056409,-0.007520,-0.009365,0.022910,-0.017637,-0.003958
10,841,8,DE,-0.282744,-0.700027,1.309253,-1.309253,0.248085,-2.226840,-0.248085,...,-0.034382,1.487869,-0.110837,0.433819,-0.035941,-0.464359,1.558207,-1.496966,0.454327,-0.599628


In [21]:
fr_coal_on = -0.725

In [22]:
X_train_FR['FR_COAL_ON'] = 1
X_train_FR.loc[X_train_FR['FR_COAL'] < fr_coal_on, 'FR_COAL_ON'] = 0

In [23]:
X_train_FR.head()

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,DE_GAS_COST,FR_COAL_COST,FR_GAS_COST,DE_LIGNITE_COST,DE_COAL_CARBON,DE_GAS_CARBON,FR_COAL_CARBON,FR_GAS_CARBON,DE_LIGNITE_CARBON,FR_COAL_ON
0,1057,0,FR,0.048418,-0.465067,-0.079296,0.079296,0.405549,0.650577,-0.405549,...,-0.635235,-0.007118,0.280396,-0.076148,0.362577,0.331012,0.064572,-0.146111,0.853381,1
1,1179,1,FR,0.966997,1.222131,0.331356,-0.331356,0.405549,0.778627,-0.405549,...,1.644639,-0.732682,2.947344,1.015751,0.461441,0.639310,-0.549565,1.145703,0.941156,0
2,1327,2,FR,-0.068972,-0.667390,-1.102015,1.102015,-1.080403,0.256736,1.080403,...,2.213675,-0.874976,0.826113,-0.537791,-0.189585,0.774063,-0.586965,0.288870,-0.445656,0
4,2016,3,FR,-0.134670,-0.834564,-1.051716,1.051716,-1.881881,-0.612133,1.881881,...,0.699151,0.046394,0.030499,0.010931,0.017693,-0.036424,0.020011,-0.001589,0.005824,1
6,2047,5,FR,-0.297850,-0.470371,0.144615,-0.144615,-1.208286,-1.811403,1.208286,...,-0.125035,-0.421666,-0.062324,-0.153950,-0.047898,0.580054,-0.509118,0.289129,-0.229615,0


In [24]:
X_train_DE.head()

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,DE_COAL_COST,DE_GAS_COST,FR_COAL_COST,FR_GAS_COST,DE_LIGNITE_COST,DE_COAL_CARBON,DE_GAS_CARBON,FR_COAL_CARBON,FR_GAS_CARBON,DE_LIGNITE_CARBON
3,111,2,DE,-0.068972,-0.667390,-1.102015,1.102015,-1.080403,0.256736,1.080403,...,-0.282610,2.213675,-0.874976,0.826113,-0.537791,-0.189585,0.774063,-0.586965,0.288870,-0.445656
5,800,3,DE,-0.134670,-0.834564,-1.051716,1.051716,-1.881881,-0.612133,1.881881,...,0.041018,0.699151,0.046394,0.030499,0.010931,0.017693,-0.036424,0.020011,-0.001589,0.005824
7,831,5,DE,-0.297850,-0.470371,0.144615,-0.144615,-1.208286,-1.811403,1.208286,...,-0.039670,-0.125035,-0.421666,-0.062324,-0.153950,-0.047898,0.580054,-0.509118,0.289129,-0.229615
9,779,7,DE,0.057599,-0.625625,0.002239,-0.002239,-0.676226,-0.745182,0.676226,...,0.132404,-0.146745,-0.403380,-0.276365,0.056409,-0.007520,-0.009365,0.022910,-0.017637,-0.003958
10,841,8,DE,-0.282744,-0.700027,1.309253,-1.309253,0.248085,-2.226840,-0.248085,...,-0.034382,1.487869,-0.110837,0.433819,-0.035941,-0.464359,1.558207,-1.496966,0.454327,-0.599628


In [25]:
def country_flow(data):
	"""
	Transforms the given data by rearranging columns based on the country.
	"""
	df = data.copy()
	df['EXCHANGE'] = np.where(df['COUNTRY'] == 'DE', df['DE_FR_EXCHANGE'], df['FR_DE_EXCHANGE'])
	df['SELF_EXPORT'] = np.where(df['COUNTRY'] == 'DE', df['DE_NET_EXPORT'], df['FR_NET_EXPORT'])
	df['OTHER_EXPORT'] = np.where(df['COUNTRY'] == 'DE', df['FR_NET_EXPORT'], df['DE_NET_EXPORT'])
	df['SELF_CONSUMPTION'] = np.where(df['COUNTRY'] == 'DE', df['DE_CONSUMPTION'], df['FR_CONSUMPTION'])
	df['OTHER_CONSUMPTION'] = np.where(df['COUNTRY'] == 'DE', df['FR_CONSUMPTION'], df['DE_CONSUMPTION'])
	df = df.drop(['DE_CONSUMPTION', 'FR_CONSUMPTION', 'DE_FR_EXCHANGE', 'FR_DE_EXCHANGE', 'DE_NET_EXPORT',
				  'DE_NET_IMPORT', 'FR_NET_EXPORT', 'FR_NET_IMPORT'], axis=1)
	return df

In [26]:
X_train_FR = country_flow(X_train_FR)
X_train_DE = country_flow(X_train_DE)

X_train_FR.head()

,ID,DAY_ID,COUNTRY,DE_GAS,FR_GAS,DE_COAL,FR_COAL,DE_HYDRO,FR_HYDRO,DE_NUCLEAR,...,DE_GAS_CARBON,FR_COAL_CARBON,FR_GAS_CARBON,DE_LIGNITE_CARBON,FR_COAL_ON,EXCHANGE,SELF_EXPORT,OTHER_EXPORT,SELF_CONSUMPTION,OTHER_CONSUMPTION
0,1057,0,FR,0.592572,-0.261565,0.534535,0.095197,1.312903,0.677089,0.470981,...,0.331012,0.064572,-0.146111,0.853381,1,0.079296,0.650577,0.405549,-0.465067,0.048418
1,1179,1,FR,1.111008,1.991028,0.660390,-0.786509,-0.681081,0.709616,0.830659,...,0.639310,-0.549565,1.145703,0.941156,0,-0.331356,0.778627,0.405549,1.222131,0.966997
2,1327,2,FR,1.228079,0.458302,-0.247704,-0.766904,1.785758,-0.930172,0.064726,...,0.774063,-0.586965,0.288870,-0.445656,0,1.102015,0.256736,-1.080403,-0.667390,-0.068972
4,2016,3,FR,1.588542,0.069297,-0.635452,-0.718729,1.994144,-0.383690,-2.002323,...,-0.036424,0.020011,-0.001589,0.005824,1,1.051716,-0.612133,-1.881881,-0.834564,-0.134670
6,2047,5,FR,1.059828,0.528273,-0.072071,-0.766063,1.275857,-0.398178,-1.875681,...,0.580054,-0.509118,0.289129,-0.229615,0,-0.144615,-1.811403,-1.208286,-0.470371,-0.297850


In [27]:
X_train_DE.head()

,ID,DAY_ID,COUNTRY,DE_GAS,FR_GAS,DE_COAL,FR_COAL,DE_HYDRO,FR_HYDRO,DE_NUCLEAR,...,DE_COAL_CARBON,DE_GAS_CARBON,FR_COAL_CARBON,FR_GAS_CARBON,DE_LIGNITE_CARBON,EXCHANGE,SELF_EXPORT,OTHER_EXPORT,SELF_CONSUMPTION,OTHER_CONSUMPTION
3,111,2,DE,1.228079,0.458302,-0.247704,-0.766904,1.785758,-0.930172,0.064726,...,-0.189585,0.774063,-0.586965,0.288870,-0.445656,-1.102015,-1.080403,0.256736,-0.068972,-0.667390
5,800,3,DE,1.588542,0.069297,-0.635452,-0.718729,1.994144,-0.383690,-2.002323,...,0.017693,-0.036424,0.020011,-0.001589,0.005824,-1.051716,-1.881881,-0.612133,-0.134670,-0.834564
7,831,5,DE,1.059828,0.528273,-0.072071,-0.766063,1.275857,-0.398178,-1.875681,...,-0.047898,0.580054,-0.509118,0.289129,-0.229615,0.144615,-1.208286,-1.811403,-0.297850,-0.470371
9,779,7,DE,0.386191,0.727314,0.255380,-0.778036,0.281094,-0.739291,-1.887303,...,-0.007520,-0.009365,0.022910,-0.017637,-0.003958,0.002239,-0.676226,-0.745182,0.057599,-0.625625
10,841,8,DE,0.955400,0.278566,-0.234473,-0.755877,0.385716,-0.580103,-1.889094,...,-0.464359,1.558207,-1.496966,0.454327,-0.599628,1.309253,0.248085,-2.226840,-0.282744,-0.700027


In [28]:
#export the data to csv
X_train_FR.to_csv('X_train_FR.csv', index=False)
X_train_DE.to_csv('X_train_DE.csv', index=False)